In [5]:
import pandas as pd
import numpy as np

In [6]:
ttemp       = pd.read_csv("team_att_w_cluster.csv")  
ttemp['date']  = pd.to_datetime(ttemp['date'])  # add ending date/time for which team info is valid
team_att    = ttemp.assign(next_date=ttemp.groupby('team_id').date.shift(-1))
match       = pd.read_csv("match_discrete_htwt.csv")
match['date']  = pd.to_datetime(match['date'])
player_att  = pd.read_csv("player_att_with_endtime.csv")

cont_att    = [att for att in player_att.loc[:,'overall_rating':'next_date'].select_dtypes("float64")]
disc_att    = [ 'preferred_foot_right',
                'preferred_foot_left',
                'attacking_work_rate_medium',
                'attacking_work_rate_high',
                'attacking_work_rate_low',
                'defensive_work_rate_medium',
                'defensive_work_rate_high',
                'defensive_work_rate_low' ]
htwt        = ['height', 'weight']
atts        = cont_att + disc_att + htwt

# Update as hometeam 
for att in atts:
    team_att['h_' + att] = ''
h_atts = ['h_' + att for att in atts]

# Update as awayteam
for att in atts:
    team_att['a_' + att] = ''
a_atts = ['a_' + att for att in atts]

In [7]:
def collate_h_info(row):
    date = row['next_date']
    h_matches = match[match['home_team_id'] == row['team_id']]
    h_matches = h_matches[(h_matches['date'] >= row['date']) & ((date != date) | (h_matches['date'] < date))]
    h_matches = h_matches[h_atts]
    return h_matches.mean(axis=0)
    
def collate_a_info(row):
    date = row['next_date']
    a_matches = match[match['away_team_id'] == row['team_id']]
    a_matches = a_matches[(a_matches['date'] >= row['date']) & ((date != date) | (a_matches['date'] < date))]
    a_matches = a_matches[a_atts]
    return a_matches.mean(axis=0)

In [8]:
team_att[h_atts] = team_att.apply(collate_h_info, axis=1, result_type='expand')
team_att[a_atts] = team_att.apply(collate_a_info, axis=1, result_type='expand')
team_att.to_csv("team_att_cluster_endtime_collated.csv")